# Автогенерация текстовых описаний к видео (кейс Rutube)

В данном кейсе вам предлагается решить задачу автогенерации краткого тектового описания к видео, на основе видеофайла и автоматической транскрибации.

Структура датасета следующая:

train.csv
- **video_name** - название видео (в директории **train_video**)
- **stt_name** - название файла с транскрибацией (в директории **train_stt**)
- **category_name** - категория видео
- **title** - название видео
- **description** - описание видео

В ноутбуке вы можете пронаблюдать baseline модель, без обучения (unsupervised) в качестве простого примера, основанную только на файле транскрибации. Также в конце считается метрика meteor по baseline модели и модели, которая из транскрипта речи (STT) выдает первые несколько предложений для сравнения.

Тестовый датасет будет прислан вам позднее, поэтому здесь он фигурировать не будет.

Немного про модель: LexRankSummarizer, не вдаваясь в детали, можно сказать, что модель основана на статистиках, ее цель - найти самые "важные" предложения из полного текста (STT).

Предложения представляются в виде мешка слов и получают эмбеддинги c tfidf, далее считаются косинусные близости предложений друг с другом. Следующая часть модели взята из немалоизвестной PageRank - строится граф, где на рёбрах стоит косинусная близость. Финальная часть  - по графу строится матрица, в ней находится максимальное сингулярное значение и таким образом находятся самые "значимые" предложения из большого текста.

Подробнее можно почитать например тут https://www.codingninjas.com/studio/library/lexrank

На метрики и сравнение моделей на других бенчмарках тут https://www.dialog-21.ru/media/5764/golovizninavspluskotelnikovev038.pdf

Для предобработки данных мы только удаляем стоп-слова (слишком часто встречаемые, например предлоги, союзы и тп), которые могут портить модель.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!curl -o /content/drive/MyDrive/rutube_hackathon_novosibirsk.zip -L
!unzip -q /content/drive/MyDrive/rutube_hackathon_novosibirsk.zip

curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information


In [3]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 226.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21694 sha256=347e5b75a7d6e7cbf7fad5d4cb98f3f585a4e12e79a0c81fc63c972c21b755f2
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c3b3e61ff7a68c9efbe12c09a0cb0d9548870f1ff9851b2945d4e42029b2bab4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=106

In [4]:
# lex rank - unsupervised upproach
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
import nltk
from nltk.corpus import stopwords
import numpy as np


# nltk.download('stopwords')

# Допольнительные стоп-слова можно скачать здесь
# https://github.com/stopwords-iso/stopwords-ru/blob/master/raw/stop-words-russian.txt
# но в этот список мы также добавили пару примеров вручную, поэтому прикладываем готовый файл.
# Вы также можете модифицировать на свое усмотрение, или вовсе от него отказаться

with open("/content/rutube_hackathon_novosibirsk/stop-words-russian.txt", 'r') as f:
    extra_stop_words = f.readlines()
    extra_stop_words = [line.strip() for line in extra_stop_words]


def sumy_method(text, n_sent: int = 4):

    parser = PlaintextParser.from_string(text, Tokenizer("russian"))

    stemmer = Stemmer("russian")
    summarizer = LexRankSummarizer(stemmer)
    stopwords_ru = stopwords.words('russian')
    stopwords_ru.extend(extra_stop_words)
    summarizer.stop_words = stopwords_ru

    #Summarize the document with n_sent sentences
    summary = summarizer(parser.document, n_sent)
    dp = []
    if len(summary)> 0:
        for i in summary:
            lp = str(i)
            dp.append(lp)

        final_sentence = ' '.join(dp)
    else:
        final_sentence = ''
    if len(final_sentence.split(" "))>512:
        final_sentence = " ".join(final_sentence.split(" ")[:512])
    return final_sentence

In [5]:
import pandas as pd
import os
PATH_TO_DATA = '/content/rutube_hackathon_novosibirsk/train'
dataset = pd.read_csv(os.path.join(PATH_TO_DATA, "train.csv"))

In [6]:
dataset.head(5)

,video_name,stt_name,category_name,title,description
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку..."
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...


### Для части видео речи может не быть, в бейзлайне мы это никак не учитываем, но вам предлагаем поработать и с такими ситуациями
31.mp4, 74.mp4, 111.mp4, 298.mp4, 478.mp4 - нет речи


In [6]:
dataset[dataset.video_name == '478.mp4']

,video_name,stt_name,category_name,title,description
478,478.mp4,478.txt,Путешествия,Прогулка по стране - Екатеринбург,Прогулка по Екатеринбургу — третьему по величи...


In [7]:
with open(os.path.join(PATH_TO_DATA, 'train_stt', '478.txt'), 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
lines

['[75.18s -> 1590.92s]  С вами был Игорь Негода.']

### Чтобы понять сколько предложений нам нужно выдавать в качестве описания, посчитаем статистики

In [7]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
import nltk
nltk.download('punkt')
dataset['len'] = dataset.description.apply(lambda l : len(sent_tokenize(l)))
print("Среднее число предложений в трейн датасете", np.mean(dataset['len'].to_list()))
print("Медиана", np.median(dataset['len'].to_list()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Среднее число предложений в трейн датасете 4.032
Медиана 3.0


### Теперь поймём примерный размер в токенах

In [9]:
dataset['len_tokens'] = dataset.description.apply(lambda l : len(l.split(" ")))

print("Среднее число слов в трейн датасете", np.mean(dataset['len_tokens'].to_list()))
print("Медиана", np.median(dataset['len_tokens'].to_list()))
print("Максимум", np.max(dataset['len_tokens'].to_list()))

Среднее число слов в трейн датасете 51.324
Медиана 42.0
Максимум 348


In [10]:
# поэтому в sumy_method мы добавили ограничение на число слов в сгенерированном тексте
# (512 слов в нашем случае, решили так ограничить макс 348 слов из трейна)

### Генерируем текстовые описания для всех видео из трейна по текстовому описанию (из Speech To Text)
Если в видео не было речи, то в качестве описания ставим категорию видео

In [11]:
# Очистим STT от временных кодов
from tqdm import tqdm
tqdm.pandas()
def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

In [12]:
def gen_description(stt_name, n_sent, category_name):

    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = sumy_method(lines, n_sent)
        if len(res)>0:
            return res
        else:
            return category_name


In [13]:
import nltk
nltk.download('stopwords')

dataset['stt_sum'] = np.nan
dataset['stt_sum'] = dataset.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name), axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 500/500 [09:33<00:00,  1.15s/it]


In [14]:
# видео, по которым нет речи и соответсвенно модель не смогла ничего выдать
dataset[dataset.stt_sum.isin(dataset.category_name.unique())]

,video_name,stt_name,category_name,title,description,len,len_tokens,stt_sum
31,31.mp4,31.txt,Авто-мото,DSC OFF на Байкальской миле 2020,"ГАЗ-24 «Волга КГБ» — проект Гурама Инцкирвели,...",4,48,Авто-мото
74,74.mp4,74.txt,Путешествия,Прогулка по стране - Казань,"Сегодня нам предстоит прогулка по городу, кото...",3,44,Путешествия
111,111.mp4,111.txt,Спорт,Команда MOTORCITY на Байкальской миле 2020,MOTORCITY собрала на фестиваль скорости “Байка...,5,116,Спорт
178,178.mp4,178.txt,Путешествия,Прогулка по стране - Москва - часть 1,Первый выпуск проекта “Прогулки по стране” пос...,1,23,Путешествия
212,212.mp4,212.txt,Спорт,Команда FCM Racing Team на Байкальской миле 2020,Эта команда привезла на Байкал соль Бонневилля...,5,68,Спорт
298,298.mp4,298.txt,Авто-мото,IVECO грузовик на Байкальской миле 2020,"Фестиваль скорости ""Байкальская Миля 2020"" соб...",4,47,Авто-мото


In [15]:
dataset.head(3)

,video_name,stt_name,category_name,title,description,len,len_tokens,stt_sum
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,3,54,"Итак, после первого раунда третье место с резу..."
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,4,48,"Так. Короче, чтобы выходом надо сделать задани..."
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...",5,60,"Короче, Авет, сколько всего произошло на прошл..."


In [16]:
dataset.stt_sum.to_list()[:3]

['Итак, после первого раунда третье место с результатом два балла у нас занимает Кика. Картина стоит в долларах, не в рублях. Давай, делаем вторую картину. Не переживай, меня это не волнует ни разу, так что давай делать.',
 'Так. Короче, чтобы выходом надо сделать задание. Но с этими заданиями это вообще жопа. Что с Аветом, я не понимаю, он грузит видос уже где-то минут 25 и не может загрузить. Короче, давай одна минута.',
 'Короче, Авет, сколько всего произошло на прошлой неделе ты даже не в курсе. Потому что у тебя на лице и так черные точки. Ее надо типа убивать, да, вообще? А зачем мы это смотрим?']

In [17]:
# dataset.to_csv("train_with_generated_sum.csv")

### Посчитаем метрику meteor

In [18]:
def func(stt_name, text, text_sum):
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 8)
    else:
        return 0
dataset['meteor'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.stt_sum), axis=1)

In [19]:
print("Значение метрики meteor для unsupervised модели", dataset.meteor.mean())

Значение метрики meteor для unsupervised модели 0.09976261646000001


In [20]:
# метрика в данной реализации имеет значения от 0 до 1

### Сравним с моделью, которая выдает первые 4 предложения из STT

In [21]:
%%time
def func(stt_name, text, category_name):
    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        res = lines[:4]
    res = " ".join(lines)
    if isinstance(res, str):
        return round(meteor([word_tokenize(text)],word_tokenize(res)), 8)
    else:
        return round(meteor([word_tokenize(text)],word_tokenize(category_name)), 8)
dataset['meteor_first4'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.category_name), axis=1)

CPU times: user 57.8 s, sys: 154 ms, total: 58 s
Wall time: 58.4 s


In [22]:
print("Значение метрики meteor для модели, выдающей первые 4 предложения", dataset.meteor_first4.mean())

Значение метрики meteor для модели, выдающей первые 4 предложения 0.057717922639999995


In [23]:
PATH_TO_DATATEST = '/content/drive/MyDrive/test'
dataset_test = pd.read_csv(os.path.join(PATH_TO_DATATEST, "test.csv"))

In [24]:
def gen_description2(stt_name, n_sent, category_name):

    with open(os.path.join(PATH_TO_DATATEST, 'test_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = sumy_method(lines, n_sent)
        if len(res)>0:
            return res
        else:
            return category_name

In [25]:
dataset_test['stt_sum'] = np.nan
dataset_test['stt_sum'] = dataset_test.progress_apply(lambda l: gen_description2(l.stt_name, 4, l.category_name), axis=1)

100%|██████████| 100/100 [02:38<00:00,  1.58s/it]


In [26]:
dataset_test[dataset_test.stt_sum.isin(dataset_test.category_name.unique())]

,video_name,stt_name,category_name,stt_sum


In [27]:
dataset_test

,video_name,stt_name,category_name,stt_sum
0,0.mp4,0.txt,Новости и СМИ,"А этот пёс считает, что футбольное поле отличн..."
1,1.mp4,1.txt,Развлечения,"Мама очень круто готовит, мне всегда очень нра..."
2,2.mp4,2.txt,Развлечения,"Ну как сказать, если все будут говорить правду..."
3,3.mp4,3.txt,Видеоигры,"Ну слушай, ты в какой-то момент изначально, да..."
4,4.mp4,4.txt,Развлечения,"Мы приходим к подписчику, жестко убираемся у н..."
...,...,...,...,...
95,95.mp4,95.txt,Развлечения,Кто из вас первые прошьет два раза нашего врат...
96,96.mp4,96.txt,Юмор,а можно не быть таким дрищом привет говна с то...
97,97.mp4,97.txt,Спорт,"И давайте сделаем, чтобы последний из-за спины..."
98,98.mp4,98.txt,Блоги,Смотрим их контент и пытаемся доказать в течен...


In [28]:
itog = dataset_test.drop(columns=["stt_name", "category_name"])

In [29]:
itog

,video_name,stt_sum
0,0.mp4,"А этот пёс считает, что футбольное поле отличн..."
1,1.mp4,"Мама очень круто готовит, мне всегда очень нра..."
2,2.mp4,"Ну как сказать, если все будут говорить правду..."
3,3.mp4,"Ну слушай, ты в какой-то момент изначально, да..."
4,4.mp4,"Мы приходим к подписчику, жестко убираемся у н..."
...,...,...
95,95.mp4,Кто из вас первые прошьет два раза нашего врат...
96,96.mp4,а можно не быть таким дрищом привет говна с то...
97,97.mp4,"И давайте сделаем, чтобы последний из-за спины..."
98,98.mp4,Смотрим их контент и пытаемся доказать в течен...


In [30]:
itog.to_csv('test_submit.csv')